In [1]:
#compare functions

In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import time 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


np.set_printoptions(precision=20)

def fpowi(x,n): #custom positive integer power function n>=2
    f=x*x
    for i in range(n-2):
        f=f*x
    if n>0:
        return f
    if n<0:
        return 1/f

def fpowf(x,y): #custom float power function twice as fast
    return np.exp(y*np.log(x)) 

In [3]:

# Re=4000-1e8
# eps/D=0-0.05

def create_(N1,type_):
    if type_==1:
        xnorm_=np.arange(0,1+1/N1,1/N1)
        xv, yv=np.meshgrid(xnorm_,xnorm_)
        xy=np.column_stack((np.ndarray.flatten(xv),np.ndarray.flatten(yv)))
    if type_==2:
        xy=np.random.rand((N1+1)*(N1+1),2)
        
    xy[:,0]=4000*np.power(1e8/4000,xy[:,0]) #geometric series
    #xy[:,0]=np.ones(len(xy))*4000+(1e8-4000)*xy[:,0] #arithmetic series
    #xy[:,1]=0.05*xy[:,1] #arithmetic series
    xy[:,1]=1e-6*np.power(0.05/1e-6,xy[:,1]) #geometric series
    return xy


def z_calc(xy,iter_):
    z_=0.1*np.ones(len(xy))
    y1=xy[:,1]/3.7
    y2=2.51/xy[:,0]
    for i in range(iter_):
        y3=np.log(y2/np.sqrt(z_)+y1)
        z_=1.3254745276195998/(y3*y3)  #0.25*ln(10)^2
    return z_

def z_err000(xy,z):
    err_=1/np.sqrt(z_)+2*np.log10(2.51/(xy[:,0]*np.sqrt(z_))+xy[:,1]/3.7)
    print('norm 2=',np.linalg.norm(err_))
    print('max err=',np.amax(abs(err_)))
    print('avg err=',np.ndarray.mean(abs(err_)))


In [4]:
N1=3164-1
#N1=1000-1
#N1=317-1
#N1=100-1
#N1=32-1
##N1=10-1

#training data
t0=time.time()

xy=create_(N1,1)
z_=z_calc(xy,30)

z_err000(xy,z_)

print('time to create=',round(time.time()-t0,2),'s')

norm 2= 3.856600256273761e-12
max err= 5.329070518200751e-15
avg err= 9.329905517927098e-16
time to create= 11.68 s


In [10]:
len(z_)
z_[0],z_[len(z_)-1]
#3164-1*3164-1
#numpy  =   0.039908029446170674  0.07155090409108328
#matlab =   0.039908029446171     0.071550904091083

(0.039908029446170674, 0.07155090409108328)

In [5]:
def err_(z_,z_hat,print_):
    zz=np.abs(z_-z_hat)/z_
    a1= np.max(zz)*100
    a2= np.average(zz)*100
    a3= np.sqrt(np.average(np.power(zz,2)))*100
    if print_==1:
        print('MaxRelErr=',a1,'%, MeanRelErr=',a2,'%, RootMeanSquRelErr=',a3,'%', sep='')
    return [a1,a2,a3]

def err_f(bb):
    xy=bb[0]
    z_=bb[1]
    type_=bb[2]
    z_hat=ff(xy,type_,0)
    error_=np.average(np.power(abs(z_-z_hat)/z_,2))*100
    return error_


In [6]:
-2/np.log(10)

-0.8685889638065035

In [7]:
#comparing methods

def ff(xy,type_,print_):
    Re=xy[:,0]
    epsD=xy[:,1]
    name_=''
    
    counter_=0;
    
    f=np.zeros(len(xy))
 


    #0
    if type_==counter_:
        name_='colebrook white-accepted design formula (5 iter)'
        f=0.01
        y1=epsD/3.7
        y2=2.51/Re
        #-0.8685889638065035=-2/np.log(10)
        for i in range(2):
            f=fpowi(-0.8685889638065035*np.log(y1+y2/np.sqrt(f)),-2)

    #1   
    counter_+=1
    if type_==counter_:
        name_='Swamee-Jain (Bhave, 1991 - Epanet)'

        #f=0.25/fpowf(np.log10(epsD/3.7+5.74/fpowf(Re,0.9)),2)

        w=Re*np.pi/4
        A8 = 4.61841319859066668690e+00    #5.74*(PI/4)^.9
        A9 = -8.68588963806503655300e-01   #-2/ln(10)
        y1=A8/fpowf(w,0.9)
        y2 = epsD / 3.7 + y1
        y3 = A9 * np.log(y2)
        f = 1.0 / (y3*y3)

            
    #2
    counter_+=1
    if type_==counter_:
        name_='Haaland 1983 - mentioned in white & munson fluid mechanics books'
        f=0.25/fpowi(np.log10(epsD/3.7+fpowf(7/Re,0.9)),2) #Eck 1973    

    #3
    counter_+=1
    if type_==counter_:
        name_='Goudar–Sonnad'
        a_=2/np.log(10)
        b_=epsD/3.7
        d_=np.log(10)*Re/5.02
        s_=b_*d_+np.log(d_)
        q_=fpowf(s_,s_/(s_+1))
        g_=b_*d_+np.log(d_/q_)
        z_=np.log(q_/g_)
        dla_=z_*g_/(g_+1)
        dcfa_=dla_*(1+(z_/2)/(fpowi(g_+1,2)+z_/3*(2*g_-1)))
        f=fpowi(a_*(np.log(d_/q_)+dcfa_),-2)

    #4
    counter_+=1
    if type_==counter_:
        name_='Barr 1981'
        y1=epsD/3.7+4.518*np.log10(Re/7)/(Re*(1+fpowf(Re,0.52)/29*fpowf(epsD,0.7)))
        f=fpowi(-2*np.log10(y1),-2)

    #5
    counter_+=1
    if type_==counter_:
        name_='Zigrang & Sylvester 1982 1'
        y1=epsD/3.7-5.02/Re*np.log10(epsD/3.7-5.02/Re*np.log10(epsD/3.7+13/Re))
        f=fpowi(-2*np.log10(y1),-2)
        f=0.25*fpowi(np.log10(2.51/(Re*np.sqrt(f))+epsD/3.7),-2)

    #6
    counter_+=1
    if type_==counter_:
        name_='Serghides 1984'
        y1=epsD/3.7
        y2=-2*np.log10(y1+12/Re)
        y3=-2*np.log10(y1+2.51*y2/Re)
        y4=-2*np.log10(y1+2.51*y3/Re)
        f=fpowi(y2-fpowi(y3-y2,2)/(y4-2*y3+y2),-2)        

    #7
    counter_+=1
    if type_==counter_:
        name_= 'S.Alashkar 2012 '
        y1=epsD/3.7065
        y2=2.5226/Re
        f=1.325474505*fpowi(np.log(y1-0.8686068432*y2*np.log(y1-0.8784893582*y2*np.log(y1+fpowf(1.665368035*y2,0.8373492157)))),-2)   

    #8
    counter_+=1
    if type_==counter_:
        name_= 'Vatankhah, Kouchakzadeh 2008 '
        y1=0.124*Re*epsD+np.log(0.4587*Re)
        f=fpowi(0.8686*np.log(0.4587*Re/(fpowf(y1-0.31,y1/(y1+0.9633)))),-2)

    #9
    counter_+=1
    if type_==counter_: 
        name_='Fang 2011'
        y1=0.234*fpowf(epsD,1.1007)
        y2=-60.525*fpowf(Re,-1.1105)
        y3=56.291*fpowf(Re,-1.0712)
        f=1.613*fpowi(np.log(y1+y2+y3),-2)

    #10
    counter_+=1
    if type_==counter_: 
        name_='Evangelides, Papaevangelou, Tzimopoulos 2010'
        y1=0.2479-0.0000947*fpowi(7-np.log10(Re),4)
        y2=np.log10(epsD/3.615+7.366*fpowf(Re,-0.9142))
        f=y1*fpowi(y2,-2)
        
    #11
    counter_+=1
    if type_==counter_: 
        name_='vatankhah 2019 discussion-Eq8'
        y1=0.8685889638065035 #2/np.log(10)  -a
        y2=0.39840637450199207 #1/2.51  -a1
        y3=0.12363404028060514  #   a2
        y4=y3*Re*epsD+np.log(Re/2.51) #s
        y5=1+0.49959/((y4+1)/np.log(y1*y4)-(1+5.448*y4)/(8*(y4+1))) #lambda
        f=fpowi(y1*np.log(y2*Re)-y1*y4/(y4+y5)*np.log(y1*y4),-2)
        
    #12
    counter_+=1
    if type_==counter_: 
        name_='vatankhah 2018'      
        y1=0.12363*Re*epsD+np.log(0.3984*Re)
        y2=(1+y1)/(0.5*np.log(0.8686*y1))
        y3=(1+4*y1)/(3*(1+y1))
        y4=1+1/(y2-y3)
        y6=0.3984*Re/fpowf(0.8686*y1,y1/(y1+y4))
        f=fpowi(0.8686*np.log(y6),-2)

    #13
    counter_+=1
    if type_==counter_: 
        name_='vatankhah 2019 discussion-Eq11'
        y1=0.8685889638065035 #2/np.log(10)  -a
        y2=0.39840637450199207 #1/2.51  -a1
        y3=0.12363404028060514  #   a2
        y4=y3*Re*epsD+np.log(Re/2.51) #s
        y5=(fpowi(y4,2)+2.1061*y4+3.2516)/(fpowi(y4,2)+3.1142*y4+7.0061)
        f=fpowi(y1*np.log(y2*Re/fpowf(y1*y4,y5)),-2)
 
    #Romeo, Royo and Monzón approximation
     
        
    print('\n',name_) if print_==1 else 1
    return f

#vectorized numpy code but not parallelized
#mpi4py can help you with that but it's too hard.



In [8]:
print('vector size=',np.power(N1+1,2))
#for type_ in range(9+1):
for type_ in range(13):
    z_hat=ff(xy,type_,1)
    %timeit ff(xy,type_,0)
    err_(z_,z_hat,1); # max - average - rmse

vector size= 1000000

 colebrook white-accepted design formula (5 iter)
76.9 ms ± 864 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
MaxRelErr=4.29498814671884%, MeanRelErr=0.5346164930142575%, RootMeanSquRelErr=1.0832124857760292%

 Swamee-Jain (Bhave, 1991 - Epanet)
70.7 ms ± 326 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
MaxRelErr=3.3582616174209474%, MeanRelErr=0.4959406877614587%, RootMeanSquRelErr=0.70126660650509%

 Haaland 1983 - mentioned in white & munson fluid mechanics books
63.9 ms ± 310 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
MaxRelErr=3.4192265678605276%, MeanRelErr=0.5054411273086297%, RootMeanSquRelErr=0.7228978479920696%

 Goudar–Sonnad
191 ms ± 7.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
MaxRelErr=1.0369724850886054e-10%, MeanRelErr=8.008651064371575e-12%, RootMeanSquRelErr=2.055086246725242e-11%

 Barr 1981
131 ms ± 848 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
MaxRelErr=0.5338182594059436%, MeanRelE

In [9]:
# vector size= 1024
															#mesh size								
#															1000*1000=1mill		3164*3164=10mill

#  2 iter colebrook,f0=0.01															0.79s	4.29%		
#  3 iter colebrook,f0=0.01															1.14s	0.77%		
#  5 iter colebrook, f0=0.01						32		0.2s 		0.76%		1.83s	0.023%		
# 10 iter colebrook, f0=0.01						32		0.2s 		0.76%		3.52s	3.61e-6%	
# 30 iter colebrook, f0=0.01						32		0.2s 		0.76%		10.6s	5.37e-14%	

#  Haaland 1983  (fluid mechanics books)					0.1s 		3%			0.66s	3.42%		
#  swamee jain (epanet-manual)														0.73s	3.36%		
#  swamee jain (epanet 2.2-implmentation)											0.73s	3.36%		

#  Evangelides, Papaevangelou, Tzimopoulos 2010		31		0.2s 		0.8%		1.01s	0.81%		
#  Barr 1981 										33		0.2s 		0.5%		1.35s	0.53%		
#  Fang 2011 										34		0.2s 		0.49%		1.24s	0.49%		
#  S.Alashkar 2012 									36		0.1s 		0.11%		1.17s	0.12%		
#  Zigrang & Sylvester 1982 1						37		0.1s 		0.1%		1.56s	0.11%		
# vatankhah 2018																	1.53s	0.13%				
# vatankhah 2019-eq8																1.45s	0.13%			
# vatankhah 2019-eq11																1.53s	0.13%			

#  Vatankhah, Kouchakzadeh 2008  					38		0.1s 		0.05%		0.98s	0.052%		 great
#  Serghides 1984 correction						39		0.2s 		0.003%		1.31s	0.003%		 great
#  Goudar–Sonnad 									40		0.3s 		0.00000%	1.93s	1.036e-10%	 great

In [10]:
#derivative is also needed  df/dRe

In [11]:
# %timeit np.sqrt(z_) #42 ms, 1sqrt
# aa=np.sqrt(z_)
# %timeit Re*aa #42 ms, 1mult
# bb=Re*aa
# %timeit 2.51/(Re*aa)  #87 ms 1mult,1div
# %timeit 2.51/bb #42 ms, 1 div

# aa=epsD/3.7+2.51/(Re*np.sqrt(z_))
# %timeit np.log10(aa) #132 ms  1 log10
# %timeit np.log(aa) #114 ms   1 Ln
# %timeit 0.43429448190325176*np.log(aa)  #147 1multiple, 1ln


# aa=-2*np.log10(epsD/3.7+2.51/(Re*np.sqrt(z_)))
# %timeit np.power(aa,-2) #506ms
# %timeit np.power(aa,2) #506ms
# %timeit 1/np.power(aa,2) #506ms
# %timeit 1/(aa*aa) #75ms

# %timeit  1/fpowi(aa,2) #75 ms
# %timeit  1/fpowi(aa,3) #113 ms
# %timeit  fpowii(aa,-2) #73 ms
# %timeit  fpowi(aa,4) #114ms
# %timeit  np.power(aa,4) #529ms
# %timeit  np.power(4,z_) #521ms
# %timeit  fpowf(4,z_) #160ms

# %timeit fpowf(4,z_) #160ms
# #%timeit np.power(4,z_) #514ms


# %timeit np.power(epsD,1.1007) #504ms
# %timeit fpow(epsD,1.1007) #276ms

# %timeit min(z_) #1.4s
# %timeit np.min(z_) #6 ms


# %timeit np.where(z_>0.01,0.01,0) #40 ms

# %timeit abs(z_)

In [12]:
#1 multiply = 1 division = 1 sum
#1 log_e = 4 multiply
#pow_integer = 2.5 multiply
#pow_float = 1exp+2multiply+1log= 6 multiply
#pow_exp = 4 multiply

In [26]:


%timeit np.exp(z_)

127 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
%timeit np.power(z_,2,dtype=np.float64)


509 ms ± 2.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit np.power(z_,2,dtype=np.float32)
    

210 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%timeit np.log(z_,dtype=np.float32)

95 ms ± 347 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%timeit np.log(z_,dtype=np.float64)

114 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
%timeit np.log(z_)

114 ms ± 480 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit z_*z_

32.2 ms ± 535 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit np.power(z_,2.5,dtype=np.float32)

210 ms ± 653 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
%timeit np.multiply(z_,z_,dtype=np.float64)

33.3 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [36]:
%timeit np.multiply(z_,z_,dtype=np.float32)

30.1 ms ± 528 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
